In [25]:
import json
import random
import yaml
import logging
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from mesa import Agent, Model
from mesa.agent import AgentSet

# Add utils to path
sys.path.insert(0, str(Path.cwd()))

# Import config loader and logger
from utils.config import load_config, get_config
from utils.logger import setup_logging

# Load configuration
config = load_config("config.yaml")
logger = setup_logging(
    log_level=config.simulation.get("log_level", "INFO"),
    log_file=config.logging_config.get("log_file")
)

logger.info("=" * 60)
logger.info(f"Agricultural Agent-Based Simulation")
logger.info(f"Agents: {config.simulation.get('n_agents')}")
logger.info(f"Timesteps: {config.simulation.get('n_timesteps')}")
logger.info(f"Random Seed: {config.simulation.get('random_seed')}")
logger.info("=" * 60)

# Set random seed for reproducibility
random.seed(config.simulation.get("random_seed", 42))

# Load data files from config
data_paths = config.data_paths
with open(data_paths["zones_file"]) as f:
    zones_data = json.load(f)

with open(data_paths["crops_file"]) as f:
    crops_data = json.load(f)

with open(data_paths["climate_file"]) as f:
    climate_data = json.load(f)

with open(data_paths["soil_moisture_file"]) as f:
    soil_moisture = json.load(f)

cfg = get_config()
n_agents = cfg.simulation.get("n_agents", 100)
n_timesteps = cfg.simulation.get("n_timesteps", 10)

logger.info(f"Loaded {len(zones_data['zones'])} zones")
logger.info(f"Loaded {len(crops_data['crops'])} crops")
logger.info(f"Loaded climate profiles for all zones")
logger.info(f"Loaded soil moisture data")

2026-02-14 06:30:34,413 - agroai_sim - INFO - Logging to logs\simulation.log
2026-02-14 06:30:34,419 - agroai_sim - INFO - ============================================================
2026-02-14 06:30:34,424 - agroai_sim - INFO - Agricultural Agent-Based Simulation
2026-02-14 06:30:34,434 - agroai_sim - INFO - Agents: 100
2026-02-14 06:30:34,435 - agroai_sim - INFO - Timesteps: 30
2026-02-14 06:30:34,444 - agroai_sim - INFO - Random Seed: 42
2026-02-14 06:30:34,449 - agroai_sim - INFO - ============================================================
2026-02-14 06:30:34,479 - agroai_sim - INFO - Loaded 6 zones
2026-02-14 06:30:34,484 - agroai_sim - INFO - Loaded 24 crops
2026-02-14 06:30:34,485 - agroai_sim - INFO - Loaded climate profiles for all zones
2026-02-14 06:30:34,492 - agroai_sim - INFO - Loaded soil moisture data


In [26]:
class Zone:
    def __init__(self, zone_id, data, climate_profile, soil_moisture):

        self.id = zone_id
        self.name = data["name"]
        self.soil_types = data["soil_types"]
        self.dominant_soil = data["dominant_soil"]
        self.viable_crops = data["viable_crops"]
        self.agriculture_type = data["agriculture_type"]

        self.climate_profile = climate_profile

        # Soil moisture correctly extracted here
        sm_data = soil_moisture["zones"][zone_id]
        self.mean_sm = sm_data["mean"]
        self.std_sm = sm_data["std"]

        self.shared_crop_yield = {}


    

In [27]:
class FarmerAgent(Agent):
    def __init__(self, model, unique_id, zone, land_size, strategy_type):
        super().__init__(model)
        self.unique_id = unique_id
        self.zone = zone
        self.land_size = land_size
        self.strategy_type = strategy_type
        self.chosen_crop = None
        self.yield_t_ha = 0
        self.profit = 0
        self.memory = []

    def step(self):
        self.decide_crop()
        self.compute_yield()

    def decide_crop(self):
        cfg = get_config()
        available_crops = list(crops_data["crops"].keys())
        suitability_scores = {}
        for crop in available_crops:
            suitability_scores[crop] = self.compute_suitability(crop)
        
        if self.strategy_type == "SHARED":
            shared_wt = cfg.learning.get("shared_weight", 0.6)
            community = self.zone.shared_crop_yield
            if community:
                max_community_yield = max(community.values()) if community else 1.0
                for crop in available_crops:
                    if crop in community and max_community_yield > 0:
                        community_bonus = community[crop] / max_community_yield
                        suitability_scores[crop] += shared_wt * community_bonus
                    else:
                        suitability_scores[crop] *= (1.0 - shared_wt * 0.3)
        
        self.chosen_crop = max(suitability_scores, key=suitability_scores.get)

    def compute_suitability(self, crop_name):
        cfg = get_config()
        zone_weight = cfg.suitability.get("zone_weight", 0.4)
        soil_weight = cfg.suitability.get("soil_weight", 0.25)
        climate_weight = cfg.suitability.get("climate_weight", 0.25)
        moisture_weight = cfg.suitability.get("moisture_weight", 0.1)

        crop = crops_data["crops"][crop_name]
        ecocrop = crop["ecocrop"]
        score = 0.0
        weight_total = 0.0

        # 1. Zone suitability
        zone_suitable = crop.get("ideal_zones", crop.get("suitable_zones", []))
        if self.zone.id in zone_suitable:
            score += 1.0 * zone_weight
        elif self.zone.id in crop.get("marginal_zones", []):
            score += 0.5 * zone_weight
        weight_total += zone_weight

        # 2. Soil suitability
        preferred_soils = crop.get("ideal_soils", crop.get("preferred_soils", []))
        if self.zone.dominant_soil in preferred_soils:
            score += 1.0 * soil_weight
        else:
            score += 0.3 * soil_weight
        weight_total += soil_weight

        # 3. Soil Moisture suitability
        moisture_data = soil_moisture.get(self.zone.id, {})
        if moisture_data:
            moisture_score = moisture_data.get("mean_rzsm_normalized", 0.5)
            crop_moisture_pref = crop.get("moisture_preference", "medium")
            if crop_moisture_pref == "high":
                moisture_score = moisture_score
            elif crop_moisture_pref == "low":
                moisture_score = 1.0 - moisture_score
            else:
                moisture_score = 1.0 - abs(moisture_score - 0.5) * 2
            score += moisture_score * moisture_weight
        weight_total += moisture_weight

        # 4. Climate suitability (handle both _C and _c keys from crops.json)
        climate_profile = self.zone.climate_profile
        temp_score = 0.0
        precip_score = 0.0
        t_opt_min = ecocrop.get("temp_opt_min_C", ecocrop.get("temp_opt_min_c", 15))
        t_opt_max = ecocrop.get("temp_opt_max_C", ecocrop.get("temp_opt_max_c", 30))
        t_abs_min = ecocrop.get("temp_abs_min_C", ecocrop.get("temp_abs_min_c", 5))
        t_abs_max = ecocrop.get("temp_abs_max_C", ecocrop.get("temp_abs_max_c", 40))
        p_opt_min = ecocrop.get("precip_opt_min_mm", 400)
        p_opt_max = ecocrop.get("precip_opt_max_mm", 1200)
        p_abs_min = ecocrop.get("precip_abs_min_mm", 200)
        p_abs_max = ecocrop.get("precip_abs_max_mm", 2000)

        for month_key in [f"month_{m}" for m in range(1, 13)]:
            monthly = climate_profile.get(month_key, {})
            tavg = monthly.get("tavg_mean", 20)
            if t_opt_min <= tavg <= t_opt_max:
                temp_score += 1.0
            elif t_abs_min <= tavg <= t_abs_max:
                temp_score += 0.5
            
            prectot = monthly.get("prectotcorr", {"mean": 1.5})
            p_mean = prectot["mean"] * 365
            if p_opt_min <= p_mean <= p_opt_max:
                precip_score += 1.0
            elif p_abs_min <= p_mean <= p_abs_max:
                precip_score += 0.5

        temp_score /= 12
        precip_score /= 12
        climate_score = 0.5 * temp_score + 0.5 * precip_score
        score += climate_score * climate_weight
        weight_total += climate_weight

        suitability = score / weight_total
        return suitability

    def compute_yield(self):
        cfg = get_config()
        base_yield = crops_data["crops"][self.chosen_crop]["base_yield_t_ha"]
        suitability = self.compute_suitability(self.chosen_crop)
        var_min = cfg.yield_params.get("variability_min", 0.85)
        var_max = cfg.yield_params.get("variability_max", 1.15)
        variability = var_min + (var_max - var_min) * random.random()
        
        self.yield_t_ha = base_yield * suitability * variability
        self.profit = self.yield_t_ha * self.land_size
        
        self.memory.append({
            "crop": self.chosen_crop,
            "yield": self.yield_t_ha,
            "profit": self.profit
        })

In [28]:
class FarmModel(Model):
    def __init__(self, zones_data, climate_data, crops_data, soil_moisture, n_agents):
        super().__init__()

        self.crops_data = crops_data
        self.zones = {}

        for zone_id, zone_info in zones_data["zones"].items():
            climate_profile = climate_data["zones"][zone_id]["climate_profile"]
            self.zones[zone_id] = Zone(
                zone_id,
                zone_info,
                climate_profile,
                soil_moisture
            )
        
        # ================================================================
        # PAIRED DESIGN: Equal land split for fair SHARED vs INDIVIDUAL
        # For every SHARED farmer, an INDIVIDUAL farmer gets the EXACT
        # same zone and land size. The ONLY difference is strategy.
        # ================================================================
        cfg = get_config()
        
        # Ensure even number of agents for perfect pairing
        n_pairs = n_agents // 2
        actual_agents = n_pairs * 2
        if actual_agents != n_agents:
            logger.info(f"  Adjusted agent count from {n_agents} to {actual_agents} for equal pairing")
        
        agents_list = []
        agent_id = 0
        
        for i in range(n_pairs):
            # Generate ONE random zone and land size per pair
            zone = random.choice(list(self.zones.values()))
            land_size = random.uniform(1, 10)
            
            # Create SHARED farmer
            shared_agent = FarmerAgent(
                model=self,
                unique_id=agent_id,
                zone=zone,
                land_size=land_size,
                strategy_type="SHARED"
            )
            agents_list.append(shared_agent)
            agent_id += 1
            
            # Create INDIVIDUAL farmer with SAME zone & land size
            individual_agent = FarmerAgent(
                model=self,
                unique_id=agent_id,
                zone=zone,
                land_size=land_size,
                strategy_type="INDIVIDUAL"
            )
            agents_list.append(individual_agent)
            agent_id += 1
        
        self.farmers = AgentSet(agents_list)
        
        # Log the pairing summary
        shared_count = sum(1 for a in agents_list if a.strategy_type == "SHARED")
        indiv_count = sum(1 for a in agents_list if a.strategy_type == "INDIVIDUAL")
        shared_land = sum(a.land_size for a in agents_list if a.strategy_type == "SHARED")
        indiv_land = sum(a.land_size for a in agents_list if a.strategy_type == "INDIVIDUAL")
        logger.info(f"  Paired design: {shared_count} SHARED + {indiv_count} INDIVIDUAL")
        logger.info(f"  Total land — SHARED: {shared_land:.1f} ha, INDIVIDUAL: {indiv_land:.1f} ha (equal)")


    def step(self):
        self.farmers.shuffle_do("step")
        self.update_shared_knowledge()

    
    def update_shared_knowledge(self):
        for zone in self.zones.values():

            shared_agents = [
                a for a in self.farmers
                if a.zone == zone and a.strategy_type == "SHARED"
            ]

            if not shared_agents:
                zone.shared_crop_yield = {}
                continue

            crop_stats = {}

            for agent in shared_agents:
                crop = agent.chosen_crop
                crop_stats.setdefault(crop, []).append(agent.yield_t_ha)

            zone.shared_crop_yield = {
                crop: sum(yields)/len(yields)
                for crop, yields in crop_stats.items()
            }

In [29]:
import time

# Get parameters from config
n_agents = config.simulation.get("n_agents", 100)
n_timesteps = config.simulation.get("n_timesteps", 10)

logger.info(f"\n{'='*60}")
logger.info(f"Starting simulation: {n_agents} agents × {n_timesteps} timesteps")
logger.info(f"{'='*60}")

# Track execution time
start_time = time.time()

# Initialize model
model = FarmModel(zones_data, climate_data, crops_data, soil_moisture, n_agents)
logger.info(f"✓ Model initialized with {n_agents} farmers")

# Run simulation
for t in range(n_timesteps):
    model.step()
    if t % 3 == 0:
        logger.info(f"  Completed timestep {t+1}/{n_timesteps}")

elapsed = time.time() - start_time
logger.info(f"✓ Simulation completed in {elapsed:.2f} seconds")

# Print sample results
logger.info(f"\nSample Agent Results:")
logger.info(f"{'ID':<5} {'Strategy':<12} {'Zone':<15} {'Yield (t/ha)':<15} {'Profit':<10}")
logger.info("-" * 60)
for i, agent in enumerate(model.farmers):
    if i >= 5:  # Show first 5 agents
        break
    yield_val = agent.yield_t_ha
    profit_val = agent.profit
    logger.info(f"{agent.unique_id:<5} {agent.strategy_type:<12} {agent.zone.id:<15} {yield_val:<15.2f} {profit_val:<10.2f}")

2026-02-14 06:30:34,734 - agroai_sim - INFO - 
2026-02-14 06:30:34,738 - agroai_sim - INFO - Starting simulation: 100 agents × 30 timesteps
2026-02-14 06:30:34,741 - agroai_sim - INFO - ============================================================
2026-02-14 06:30:34,746 - agroai_sim - INFO -   Paired design: 50 SHARED + 50 INDIVIDUAL
2026-02-14 06:30:34,749 - agroai_sim - INFO -   Total land — SHARED: 253.4 ha, INDIVIDUAL: 253.4 ha (equal)
2026-02-14 06:30:34,751 - agroai_sim - INFO - ✓ Model initialized with 100 farmers
--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
      

In [30]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

try:
    import seaborn as sns
except ImportError:
    sns = None

output_dir = Path("results")
output_dir.mkdir(exist_ok=True)
logger.info(f"✓ Created output directory: {output_dir}")

# ============================================================================
# COLLECT RESULTS
# ============================================================================
logger.info("\n" + "="*60)
logger.info("ANALYSIS & RESULTS")
logger.info("="*60)

rows = []
agents = getattr(model, 'farmers', None) or getattr(model, 'agents', None) or []
for a in agents:
    mem = getattr(a, 'memory', [])
    if not mem:
        continue
    yields = [m['yield'] for m in mem if 'yield' in m]
    profits = [m['profit'] for m in mem if 'profit' in m]
    rows.append({
        'unique_id': getattr(a, 'unique_id', None),
        'strategy': getattr(a, 'strategy_type', None),
        'zone': getattr(getattr(a, 'zone', None), 'id', None),
        'mean_yield': sum(yields)/len(yields) if yields else None,
        'mean_profit': sum(profits)/len(profits) if profits else None,
        'yields': yields,
        'profits': profits,
    })

df = pd.DataFrame(rows)

# ============================================================================
# CALCULATE METRICS (Yield AND Profit)
# ============================================================================
avg_profit_shared = float(df[df['strategy']=='SHARED']['mean_profit'].mean())
avg_profit_individual = float(df[df['strategy']=='INDIVIDUAL']['mean_profit'].mean())

metrics = {
    "total_agents": int(len(df)),
    "shared_agents": int((df['strategy']=='SHARED').sum()),
    "individual_agents": int((df['strategy']=='INDIVIDUAL').sum()),
    "avg_yield_all": float(df['mean_yield'].mean()),
    "avg_profit_all": float(df['mean_profit'].mean()),
    "avg_yield_shared": float(df[df['strategy']=='SHARED']['mean_yield'].mean()),
    "avg_yield_individual": float(df[df['strategy']=='INDIVIDUAL']['mean_yield'].mean()),
    "yield_advantage_shared": float(df[df['strategy']=='SHARED']['mean_yield'].mean() - 
                                     df[df['strategy']=='INDIVIDUAL']['mean_yield'].mean()),
    "avg_profit_shared": avg_profit_shared,
    "avg_profit_individual": avg_profit_individual,
    "profit_advantage_shared": float(avg_profit_shared - avg_profit_individual),
}

logger.info("\n📊 KEY METRICS:")
logger.info(f"  Total Agents: {metrics['total_agents']}")
logger.info(f"  SHARED Strategy: {metrics['shared_agents']} agents ({metrics['shared_agents']/metrics['total_agents']*100:.1f}%)")
logger.info(f"  INDIVIDUAL Strategy: {metrics['individual_agents']} agents ({metrics['individual_agents']/metrics['total_agents']*100:.1f}%)")
logger.info(f"  --- YIELD ---")
logger.info(f"  Average Yield (All): {metrics['avg_yield_all']:.3f} t/ha")
logger.info(f"  Average Yield (SHARED): {metrics['avg_yield_shared']:.3f} t/ha")
logger.info(f"  Average Yield (INDIVIDUAL): {metrics['avg_yield_individual']:.3f} t/ha")
logger.info(f"  ⭐ SHARED Yield Advantage: {metrics['yield_advantage_shared']:.3f} t/ha")
logger.info(f"  --- PROFIT ---")
logger.info(f"  Average Profit (All): {metrics['avg_profit_all']:.2f}")
logger.info(f"  Average Profit (SHARED): {metrics['avg_profit_shared']:.2f}")
logger.info(f"  Average Profit (INDIVIDUAL): {metrics['avg_profit_individual']:.2f}")
logger.info(f"  💰 SHARED Profit Advantage: {metrics['profit_advantage_shared']:.2f}")

if metrics['yield_advantage_shared'] > 0 and metrics['profit_advantage_shared'] > 0:
    logger.info(f"  ✅ SHARED OUTPERFORMS on BOTH Yield AND Profit!")
else:
    logger.info(f"  ⚠️ Mixed results - check parameters")

# ============================================================================
# STATISTICS TABLE
# ============================================================================
logger.info("\n" + "="*60)
logger.info("STATISTICAL SUMMARY BY STRATEGY")
logger.info("="*60)
print("\n")
print(df.groupby('strategy')[['mean_yield','mean_profit']].describe())

# Log per-agent details
logger.info(f"\n{'ID':<5} {'Strategy':<12} {'Zone':<15} {'Yield (t/ha)':<15} {'Profit':<10}")
logger.info("-" * 60)
for _, agent in df.iterrows():
    yield_val = agent['mean_yield'] if agent['mean_yield'] is not None else 0
    profit_val = agent['mean_profit'] if agent['mean_profit'] is not None else 0
    logger.info(f"{agent['unique_id']:<5} {agent['strategy']:<12} {agent['zone']:<15} {yield_val:<15.2f} {profit_val:<10.2f}")

# ============================================================================
# CREATE VISUALIZATIONS
# ============================================================================
cfg = get_config()
viz_cfg = cfg.visualization

# Safe defaults for visualization
fig_width = viz_cfg.get("fig_size_width", 12) if viz_cfg else 12
fig_height = viz_cfg.get("fig_size_height", 6) if viz_cfg else 6
style = viz_cfg.get("plot_style", "seaborn") if viz_cfg else "seaborn"

try:
    plt.style.use(style)
except:
    try:
        plt.style.use("seaborn-v0_8")
    except:
        pass

# Figure 1: Strategy Comparison (Boxplots)
fig, axes = plt.subplots(1, 2, figsize=(fig_width, fig_height))
fig.suptitle('Strategy Comparison: INDIVIDUAL vs SHARED', fontsize=14, fontweight='bold')

if sns is not None:
    sns.boxplot(x='strategy', y='mean_yield', data=df, ax=axes[0], palette="Set2")
    axes[0].set_title('Yield by Strategy')
    axes[0].set_ylabel('Mean Yield (t/ha)')
    axes[0].set_xlabel('Farmer Strategy')
    
    sns.boxplot(x='strategy', y='mean_profit', data=df, ax=axes[1], palette="Set2")
    axes[1].set_title('Profit by Strategy')
    axes[1].set_ylabel('Profit (units)')
    axes[1].set_xlabel('Farmer Strategy')
else:
    df.boxplot(column=['mean_yield'], by='strategy', rot=45, ax=axes[0])
    axes[0].set_title('Yield by Strategy')
    df.boxplot(column=['mean_profit'], by='strategy', rot=45, ax=axes[1])
    axes[1].set_title('Profit by Strategy')

plt.tight_layout()
plt.savefig(output_dir / '01_strategy_comparison.png', dpi=100, bbox_inches='tight')
plt.show()
logger.info("✓ Saved: 01_strategy_comparison.png")

# Figure 2: Yield & Profit Distribution (Violin Plots)
fig, axes = plt.subplots(1, 2, figsize=(fig_width, fig_height))
fig.suptitle('Distribution Shapes: Yield vs Profit', fontsize=14, fontweight='bold')

if sns is not None:
    sns.violinplot(x='strategy', y='mean_yield', data=df, ax=axes[0], palette="muted", inner='quartile')
    axes[0].set_title('Yield Distribution (Violin Plot)')
    axes[0].set_ylabel('Mean Yield (t/ha)')
    axes[0].set_xlabel('Farmer Strategy')
    
    sns.violinplot(x='strategy', y='mean_profit', data=df, ax=axes[1], palette="muted", inner='quartile')
    axes[1].set_title('Profit Distribution (Violin Plot)')
    axes[1].set_ylabel('Mean Profit')
    axes[1].set_xlabel('Farmer Strategy')
else:
    axes[0].text(0.5, 0.5, 'Seaborn required for violin plots', ha='center', va='center')
    axes[1].text(0.5, 0.5, 'Seaborn required for violin plots', ha='center', va='center')

plt.tight_layout()
plt.savefig(output_dir / '02_distribution_analysis.png', dpi=100, bbox_inches='tight')
plt.show()
logger.info("✓ Saved: 02_distribution_analysis.png")

# Figure 3: Zone Performance
fig, ax = plt.subplots(figsize=(fig_width, fig_height))
zone_perf = df.groupby('zone')['mean_yield'].mean().sort_values(ascending=False)
zone_perf.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Average Yield by Geographic Zone', fontsize=14, fontweight='bold')
ax.set_ylabel('Mean Yield (t/ha)')
ax.set_xlabel('Zone')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(output_dir / '03_zone_performance.png', dpi=100, bbox_inches='tight')
plt.show()
logger.info("✓ Saved: 03_zone_performance.png")

# Figure 4: Strategy Distribution Across Zones
fig, ax = plt.subplots(figsize=(fig_width, fig_height))
strategy_by_zone = df.groupby(['zone', 'strategy']).size().unstack(fill_value=0)
strategy_by_zone.plot(kind='bar', ax=ax, color=['#FF6B6B', '#4ECDC4'])
ax.set_title('Farmer Strategy Distribution Across Zones', fontsize=14, fontweight='bold')
ax.set_ylabel('Number of Farmers')
ax.set_xlabel('Zone')
ax.legend(title='Strategy')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(output_dir / '04_strategy_by_zone.png', dpi=100, bbox_inches='tight')
plt.show()
logger.info("✓ Saved: 04_strategy_by_zone.png")

# ============================================================================
# SAVE RESULTS - ALL WITH UTF-8 ENCODING
# ============================================================================

# Save agent results
df.to_csv(output_dir / "agents_results.csv", index=False, encoding='utf-8')
logger.info(f"✓ Saved: agents_results.csv ({len(df)} agents)")

# Save metrics as JSON
import json
with open(output_dir / "metrics.json", "w", encoding='utf-8') as f:
    json.dump(metrics, f, indent=2)
logger.info("✓ Saved: metrics.json")

# Save config
with open(output_dir / "simulation_config.yaml", "w", encoding='utf-8') as f:
    yaml.dump(config.to_dict(), f)
logger.info("✓ Saved: simulation_config.yaml")

# Save summary report
with open(output_dir / "summary_report.txt", "w", encoding="utf-8") as f:
    f.write("AGRICULTURAL AGENT-BASED SIMULATION - SUMMARY REPORT\n")
    f.write("=" * 70 + "\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Experiment: {cfg.experiment.get('name', 'baseline_run')}\n")
    f.write("\n" + "=" * 70 + "\n")
    f.write("CONFIGURATION\n")
    f.write("=" * 70 + "\n")
    f.write(f"Number of Agents: {metrics['total_agents']}\n")
    f.write(f"Simulation Steps: {n_timesteps}\n")
    f.write(f"Random Seed: {config.simulation.get('random_seed')}\n")
    f.write("\n" + "=" * 70 + "\n")
    f.write("KEY FINDINGS\n")
    f.write("=" * 70 + "\n")
    f.write(f"Average Yield (All Farmers): {metrics['avg_yield_all']:.3f} t/ha\n")
    f.write(f"Average Yield (SHARED Strategy): {metrics['avg_yield_shared']:.3f} t/ha\n")
    f.write(f"Average Yield (INDIVIDUAL Strategy): {metrics['avg_yield_individual']:.3f} t/ha\n")
    f.write(f"\n* SHARED Strategy Yield Advantage: {metrics['yield_advantage_shared']:.3f} t/ha\n")
    f.write(f"\nAverage Profit (SHARED): {metrics['avg_profit_shared']:.2f}\n")
    f.write(f"Average Profit (INDIVIDUAL): {metrics['avg_profit_individual']:.2f}\n")
    f.write(f"* SHARED Strategy Profit Advantage: {metrics['profit_advantage_shared']:.2f}\n")
    
    if metrics['yield_advantage_shared'] > 0:
        pct_gain = (metrics['yield_advantage_shared'] / metrics['avg_yield_individual']) * 100
        f.write(f"   -> {pct_gain:.1f}% yield improvement compared to INDIVIDUAL strategy\n")
    else:
        pct_loss = (abs(metrics['yield_advantage_shared']) / metrics['avg_yield_individual']) * 100
        f.write(f"   -> {pct_loss:.1f}% yield loss compared to INDIVIDUAL strategy\n")
    
    if metrics['profit_advantage_shared'] > 0:
        pct_profit_gain = (metrics['profit_advantage_shared'] / metrics['avg_profit_individual']) * 100
        f.write(f"   -> {pct_profit_gain:.1f}% profit improvement compared to INDIVIDUAL strategy\n")
    else:
        pct_profit_loss = (abs(metrics['profit_advantage_shared']) / metrics['avg_profit_individual']) * 100
        f.write(f"   -> {pct_profit_loss:.1f}% profit loss compared to INDIVIDUAL strategy\n")

logger.info("✓ Saved: summary_report.txt")

2026-02-14 06:30:37,752 - agroai_sim - INFO - ✓ Created output directory: results
--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\u2713' in position 46: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel



           mean_yield                                                    \
                count      mean       std       min       25%       50%   
strategy                                                                  
INDIVIDUAL       50.0  7.189888  7.794066  0.714214  0.899044  1.066681   
SHARED           50.0  7.269942  7.895400  0.710537  0.895158  1.066350   

                                 mean_profit                                  \
                  75%        max       count       mean        std       min   
strategy                                                                       
INDIVIDUAL  16.520121  17.176481        50.0  34.623328  46.753069  0.928083   
SHARED      16.739950  17.545356        50.0  35.087991  47.522521  0.940740   

                                                       
                 25%       50%        75%         max  
strategy                                               
INDIVIDUAL  3.290134  8.200874  49.806959  160.100447

2026-02-14 06:30:38,012 - agroai_sim - INFO - 57    INDIVIDUAL   SEMI_ARID_WARM  0.87            7.69      
2026-02-14 06:30:38,016 - agroai_sim - INFO - 58    SHARED       ARID            0.72            2.51      
2026-02-14 06:30:38,019 - agroai_sim - INFO - 59    INDIVIDUAL   ARID            0.73            2.55      
2026-02-14 06:30:38,021 - agroai_sim - INFO - 60    SHARED       IRRIGATED_SOUSS 16.61           141.28    
2026-02-14 06:30:38,024 - agroai_sim - INFO - 61    INDIVIDUAL   IRRIGATED_SOUSS 16.04           136.41    
2026-02-14 06:30:38,025 - agroai_sim - INFO - 62    SHARED       SEMI_ARID_WARM  0.91            3.95      
2026-02-14 06:30:38,025 - agroai_sim - INFO - 63    INDIVIDUAL   SEMI_ARID_WARM  0.92            3.97      
2026-02-14 06:30:38,031 - agroai_sim - INFO - 64    SHARED       SEMI_ARID_WARM  0.90            6.29      
2026-02-14 06:30:38,035 - agroai_sim - INFO - 65    INDIVIDUAL   SEMI_ARID_WARM  0.93            6.52      
2026-02-14 06:30:38,036 - ag

In [31]:
# ============================================================================
# 🌐 LAUNCH INTERACTIVE SOLARA DASHBOARD
# ============================================================================
logger.info("\n" + "="*60)
logger.info("🚀 LAUNCH INTERACTIVE DASHBOARD")
logger.info("="*60)

try:
    import subprocess
    import webbrowser
    from pathlib import Path
    
    dashboard_file = Path("dashboard_solara.py")
    
    if dashboard_file.exists():
        logger.info("✓ Solara dashboard file found")
        logger.info("📡 Starting Solara server...")
        logger.info("🌐 Dashboard will be available at: http://localhost:8000")
        logger.info("")
        logger.info("💡 TIP: The dashboard will auto-load your simulation results!")
        logger.info("    - Interactive plots with filtering")
        logger.info("    - Performance metrics overview") 
        logger.info("    - Zone analysis and distributions")
        logger.info("    - Raw data explorer")
        logger.info("")
        logger.info("⚠️  To start the dashboard, run in terminal:")
        logger.info("   solara run dashboard_solara.py")
        logger.info("")
        logger.info("🔄 Or uncomment the lines below to auto-launch:")
        
        # Uncomment these lines to auto-launch the dashboard
        # logger.info("🚀 Auto-launching dashboard...")
        # subprocess.Popen(["solara", "run", "dashboard_solara.py"], cwd=Path.cwd())
        # import time
        # time.sleep(3)  # Wait for server to start
        # webbrowser.open("http://localhost:8000")
        # logger.info("✅ Dashboard launched! Check your browser.")
        
    else:
        logger.warning("❌ dashboard_solara.py not found")
        logger.info("💡 The dashboard file should be in the project root directory")
        
except ImportError as e:
    logger.error(f"❌ Missing required package: {e}")
    logger.info("💡 Install required packages:")
    logger.info("   pip install solara solara-plotly")
except Exception as e:
    logger.error(f"❌ Error launching dashboard: {e}")

logger.info("\n" + "="*60)
logger.info("🎊 SIMULATION COMPLETE - READY FOR ANALYSIS!")
logger.info("="*60)
logger.info("")
logger.info("📊 Results Summary:")
logger.info(f"   • Total Farmers: {metrics['total_agents']}")
logger.info(f"   • SHARED Strategy: {metrics['shared_agents']} farmers ({metrics['shared_agents']/metrics['total_agents']*100:.1f}%)")
logger.info(f"   • Average Yield: {metrics['avg_yield_all']:.3f} t/ha")
logger.info(f"   • SHARED Advantage: {metrics['yield_advantage_shared']:.3f} t/ha")
logger.info("")
logger.info("📁 Generated Files:")
logger.info("   • Static plots: results/*.png") 
logger.info("   • Agent data: results/agents_results.csv")
logger.info("   • Metrics: results/metrics.json")
logger.info("   • Config backup: results/simulation_config.yaml")
logger.info("   • Summary report: results/summary_report.txt")
logger.info("")
logger.info("🌐 Interactive Dashboard:")
logger.info("   • File: dashboard_solara.py")
logger.info("   • Launch: solara run dashboard_solara.py") 
logger.info("   • Access: http://localhost:8000")
logger.info("")
logger.info("🎯 Next Steps:")
logger.info("   1. Review the static plots above")
logger.info("   2. Launch the Solara dashboard for interactive exploration")
logger.info("   3. Modify config.yaml for different experiments")
logger.info("   4. Re-run this notebook to compare results")
logger.info("")
logger.info("=" * 60)

2026-02-14 06:30:42,063 - agroai_sim - INFO - 
2026-02-14 06:30:42,065 - agroai_sim - INFO - 🚀 LAUNCH INTERACTIVE DASHBOARD
--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f680' in position 46: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\lo